In [6]:
import pandas as pd
import numpy as np
import preprocessor as p
import emoji
import emot
import  re


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
#df = pd.read_csv('../collection/data/tweet_rescrape.csv')
#bot_accounts = pd.read_csv('social_spambots_1Tw.csv')
bot_tweets = pd.concat([pd.read_csv('social_spambots_1Tw.csv'), pd.read_csv('social_spambots_2Tw.csv', encoding = "ISO-8859-1"), pd.read_csv('social_spambots_3Tw.csv')]).reset_index(drop=True)
clean_tweets = pd.read_csv('geniune_accountsTw.csv')

#bot_tweets = df[df['username'].isin(bot_accounts['username'])].reset_index(drop=True)

#clean_tweets = df[df['username'].isin(clean_accounts['username'])]



C:\Users\STUDENT\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\STUDENT\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\STUDENT\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\STUDENT\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=resul

In [8]:
#bot_tweets= bot_accounts
#clean_tweets=clean_accounts

bot_tweets['BotOrNot'] = 1
clean_tweets['BotOrNot'] = 0

combined_df = bot_tweets.append(clean_tweets, ignore_index=True)


#new_df="hgdjfhdughjkfdgkfcb"

In [9]:
#new_df= new_df[new_df['ColumnName'].notnull()]

In [10]:
#new_df = combined_df
new_df = combined_df.sample(frac=1).reset_index(drop=True)

In [11]:
#new_df = combined_df.sample(frac=1).reset_index(drop=True)
#new_df['text']="hdjkfhgfiojbhvkcjnklbfgjln b"

In [12]:
def tokenize_tweet(tweet):    
    for word in tweet.split(' '):
        if (word.isalnum() and word == word.upper() and word.isdigit() == False and len(word) > 1):
            tweet = tweet.replace(word, word + " <allcaps>")
            
    tweet = emoji.demojize(tweet, delimiters=('<', '>'))

    #p.set_options(p.OPT.HASHTAG, p.OPT.MENTION, p.OPT.NUMBER, p.OPT.RESERVED,p.OPT.URL)
    p.set_options(p.OPT.HASHTAG, p.OPT.MENTION, p.OPT.RESERVED,p.OPT.URL)
    #p.set_options()
    
    tokenized = p.tokenize(tweet)
    
    res = emot.emoticons(tokenized)     
    if 'mean' in res:             
        for i in range(len(res['mean'])):
            tokenized = tokenized.replace(res['value'][i], "<" + res['mean'][i].split(' ')[0] + ">")
    
    tokenized = tokenized.lower()
    tokenized = tokenized.replace('><', '> <')
    tokenized = tokenized.translate(str.maketrans('','',"!\"“'&(”)*+,’-./:;=?[\\]^`{|}~"))
    
    return tokenized

In [13]:
new_df= new_df[new_df['text'].notnull()]
new_df['text'] = new_df['text'].apply(lambda x: tokenize_tweet(x))

In [14]:
new_df = new_df[['text', 'retweet_count', 'user_id', 'timestamp', 'favorite_count', 'reply_count', 'BotOrNot']]

In [15]:
new_df['hashtag_count'] = new_df['text'].apply(lambda x: x.count("<hashtag>"))

In [16]:
new_df['mention_count'] = new_df['text'].apply(lambda x: x.count("<mention>"))

In [17]:
new_df['url_count'] = new_df['text'].apply(lambda x: x.count("<url>"))

In [18]:
new_df.to_csv('new_labeled_tweets1.csv', index=False)

In [19]:
contents_new_lab = pd.read_csv('new_labeled_tweets1.csv')
#contents_new_lab.head()
#contents_new_lab.tail()

In [20]:
contents_new_lab.head()

,text,retweet_count,user_id,timestamp,favorite_count,reply_count,BotOrNot,hashtag_count,mention_count,url_count
0,in the first place god made idiots that was fo...,0.0,306510948.0,2014-11-01 23:25:31,0.0,0.0,1,0,0,0
1,$mention$ doubt youre going through oklahoma b...,0.0,236449587.0,2015-04-29 15:47:02,1.0,0.0,0,0,0,0
2,you live and you learn as long as you learn fr...,0.0,231225050.0,2015-02-17 23:59:33,0.0,0.0,0,0,0,0
3,read the cool history of mustard $url$,0.0,531161755.0,2014-09-15 01:39:34,0.0,0.0,1,0,0,0
4,mi ami no ah sempre queste risposte ambigue,0.0,545306050.0,2014-09-24 13:58:55,0.0,0.0,1,0,0,0


In [21]:
contents_new_lab.tail()

,text,retweet_count,user_id,timestamp,favorite_count,reply_count,BotOrNot,hashtag_count,mention_count,url_count
6283503,$mention$ we need to go to los or romas while ...,0.0,2.320292e+09,2015-03-04 08:01:29,0.0,0.0,0,0,0,0
6283504,$mention$ i will get the mm3d but <allcaps> iw...,0.0,1.588868e+07,2014-11-05 23:44:20,0.0,0.0,0,0,0,0
6283505,$reserved$ <allcaps> $mention$ gusto talagang ...,2.0,5.491811e+08,2013-10-16 15:32:29,0.0,0.0,0,0,0,0
6283506,if the romans had been obliged to learn latin ...,0.0,1.699325e+09,2014-09-26 06:05:53,0.0,0.0,1,0,0,0
6283507,$mention$ من <allcaps> ايفون <allcaps> ٥ موقفي...,0.0,2.081475e+08,2014-11-16 16:16:19,0.0,0.0,0,0,0,0
